In [61]:
import re
frs_match = re.compile(r"^(?:([\*]*) )?([\+\* ]*)(FRS\d+)\.?([\d\.]+)?([\+\*\:]*)?\s*(.+)?$").match
#frs_match = re.compile(r"^(?:([\*]*) )?([\* ]*)(FRS\d+)\.?([\d\.]+)?([\*\:]*)?\s*(.+)?$").match
subitem_match = re.compile(r"^(?:([#]+)\s*)").match
magic_match = re.compile(r"^@lm\:\<(.+)\>").match

class NoMatch(Exception):
    pass

class NumState():
    VALID_INDENTS = "*+"
    LINE_FRS = 0
    LINE_SUBITEM = 1
    LINE_OTHER = 2
    LINE_HEADER = 3
    def __init__(self, depth=None, startval=1, indent_char="*"):
        self._startval = startval
        self.state = self._newstate()
        self.ilvl = 0
        self.nlvl = 0 
        self.clvl = 0
        self.frs = ""
        self.nums = ""
        self.fmt = ""
        self.depth = depth
        if indent_char not in self.VALID_INDENTS:
            raise ValueError("Invalid Indent Character " + indent_char)
        self.indent_char = indent_char
        self.line = ""
        
    def _newstate(self):
        return [self._startval]
        
    def _grow(self, n):
        while len(self.state) < n:
            self.state.append(self._startval)
    
    def set_state(self, lvl, val):
        self._grow(lvl)
        self.state[lvl] = val
    
    def set_state_from_nums(self, nums):
        if not nums:
            lnums = []
        else:
            lnums = nums.split(".")
        self.state = [self._startval] * len(lnums)
        for i, n in enumerate(lnums):
            self.state[i] = int(n)
        return len(lnums)
    
    def set_str(self, s):
        m = frs_match(s)
        if not m:
            raise NoMatch(s)
        return self.set_match(m)

    def set_match(self, m):
        indent, pre_fmt, frs, nums, post_fmt, bob = m.groups()
        self.ilvl = len(indent or "")
        self.fmt = pre_fmt
        self.frs = frs
        self.nlvl = self.set_state_from_nums(nums)
        self.clvl = 0
        
    def frs_string(self):
        return self.format_lvl(self.clvl)
    
    def indent(self, n=1):
        maxn = self.clvl + self.nlvl + n
        minn = self.clvl + self.nlvl
        self._grow(maxn)
        for i in range(minn, maxn):
            self.state[i] = 1
        self.clvl += n
    
    def dedent(self, n=1):
        self.clvl -= n
        self.next()
        
    def next(self):
        self.state[self.clvl+self.nlvl-1] += 1
        
    def feed(self, line):
        self.line = line
        m = frs_match(line)
        if m:
            self.set_match(m)
            return self.LINE_FRS
        m = subitem_match(line)
        if m:
            self.set_indent(len(m.group(1)))
            return self.LINE_SUBITEM
        return self.LINE_OTHER
    
    def set_indent(self, n):
        diff = n - (self.ilvl + self.clvl)
        if diff == 0:
            self.next()
        elif diff > 0:
            self.indent(diff)
        else:
            self.dedent(-diff)
            
    def curr_state(self, lvl):
        return self.state[:self.nlvl + lvl]
        
    def format_lvl(self, lvl):
        if not self.state:
            return ""
        idl = lvl + self.ilvl
        if idl < 0: idl = 0
        ind = self.indent_char * idl
        if self.depth is not None and idl > self.depth:
            return ind
        space = " " if ind else ""
        frs = self.frs
        fmt = "*"
        nums = ".".join(str(n) for n in self.curr_state(lvl))
        dot = "." if nums else ""
        post = ":*"
        return "".join((ind, space, fmt, frs, dot, nums, post))
    
    def parse_text(self, text):
        lines = text.splitlines()
        for line in lines:
            typ = self.feed(line)
            if typ == self.LINE_FRS:
                v = line
            elif typ == self.LINE_SUBITEM:
                fmt = self.frs_string()
                if not fmt:
                    v = line
                    break
                try:
                    _, tail = line.split(" ", 1)
                except ValueError:
                    v = fmt
                else:
                    v = " ".join((fmt, tail))
            elif typ == self.LINE_OTHER:
                v = line
            else:
                raise ValueError(typ)
            yield v

In [62]:
header_match("*FRS1234*").groups()

('*', 'FRS1234', '*')

In [64]:
def test_match(s):
    return frs_match(s).groups()

def assert_match(s, exp):
    m = test_match(s)
    assert m == exp, (m, exp)
    
assert_match("*FRS1234*", (None, "*", "FRS1234", None, "*", None))
assert_match("* *FRS1234*", ("*", "*", "FRS1234", None, "*", None))
assert_match("*FRS1234.5.6*", (None, "*", "FRS1234", "5.6", "*", None))
assert_match("*+FRS1234.5.6+*", (None, "*+", "FRS1234", "5.6", "+*", None))
assert_match("** *+FRS1234.5.6.7.8+*", ("**", "*+", "FRS1234", "5.6.7.8", "+*", None))
assert_match("* *FRS1234:*", ("*", "*", "FRS1234", None, ":*", None))
assert_match("* *FRS1234*:", ("*", "*", "FRS1234", None, "*:", None))
assert_match("* *FRS1234*: Bob", ("*", "*", "FRS1234", None, "*:", "Bob"))

def assert_ab(a, b):
    try:
        assert a == b, "%r != %r" % (a, b)
    except AssertionError as e:
        print(e.args)
        raise

def test_state(s):
    state = NumState()
    state.set_str(s)
    return state
    
def assert_state(s, statevals, ilvl, nlvl):
    state = test_state(s)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.state, statevals)

def test_ilvl(s, ilvl):
    state = NumState()
    state.set_str(s)
    return state
    
def assert_ilvl(s, ilvl):
    state = test_ilvl(s, ilvl)
    assert_ab(state.ilvl, ilvl)
    
def test_fmt(s):
    state = NumState()
    state.set_str(s)
    return state
    
def assert_fmt1(s, id, fmt):
    state = test_fmt(s)
    state.indent(id)
    assert_ab(state.frs_string(), fmt)
    
def assert_fmt2(s, id, fmt):
    state = test_fmt(s)
    for _ in range(id):
        state.indent(1)
    assert_ab(state.frs_string(), fmt)
    
def assert_indent(s, id, fmt):
    assert_fmt1(s, id, fmt)
    assert_fmt2(s, id, fmt)

assert_ilvl("*FRS1234*", 0)
assert_ilvl("*FRS1234.5.6*", 0)
assert_ilvl("** *FRS1234.5.6*", 2)

assert_state("*FRS1234*", [], 0, 0)
assert_state("*FRS1234.5.6*", [5, 6], 0, 2)
assert_state("** *FRS1234.5.6*", [5, 6], 2, 2)
assert_state("** *FRS1234*", [], 2, 0)

def assert_state2(s, ilvl, nlvl, clvl):
    state = test_state(s)
    st = state.state.copy()
    state.indent()
    st.append(1)
    assert_ab(state.state, st)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.clvl, clvl)

assert_state2("*FRS1234*", 0, 0, 1)
assert_state2("* *FRS1234*", 1, 0, 1)
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("* *FRS1234*", 1, "** *FRS1234.1:*")
assert_indent("*FRS1234.1", 1, "* *FRS1234.1.1:*")
assert_indent("*FRS1234.1", 2, "** *FRS1234.1.1.1:*")

def assert_next(s, exp):
    st = NumState()
    st.set_str(s)
    st.next()
    assert_ab(st.frs_string(), exp)

def assert_dedent(s, id, nxt, dd, fmt, typ=1):
    st = NumState()
    st.set_str(s)
    st.indent(id)
    for _ in range(nxt):
        st.next()
    if typ == 1:
        st.dedent(dd)
    else:
        for _ in range(dd):
            st.dedent(1)
    assert_ab(st.frs_string(), fmt)

assert_next("*FRS1234.1*", "*FRS1234.2:*")
assert_next("* *FRS1234.1*", "* *FRS1234.2:*")
assert_next("*FRS1234.1.2*", "*FRS1234.1.3:*")
assert_next("* *FRS1234.1.2*", "* *FRS1234.1.3:*")
assert_dedent("*FRS1234.1*", 1, 2, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 1, 1, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 2, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 3, 1, 2, "* *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 1, 2, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 1, 1, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 2, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 3, 1, 2, "** *FRS1234.1.2:*")

def assert_feed_indent(s, feed, exp):
    st = NumState()
    st.set_str(s)
    st.feed(feed)
    assert_ab(st.frs_string(), exp)
    
assert_feed_indent("*FRS1234*", "# Bob", "* *FRS1234.1:*")
assert_feed_indent("*FRS1234*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "# Bob", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## Bob", "** *FRS1234.1.1.1:*")
assert_feed_indent("* *FRS1234.1*", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("* *FRS1234*", "## Bob", "** *FRS1234.1:*")
assert_feed_indent("** *FRS1234.1*", "## Bob", "** *FRS1234.2:*")
assert_feed_indent("* *FRS1234.1*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("** *FRS1234.1.1", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("*FRS1234.1*", "# ", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## ", "** *FRS1234.1.1.1:*")
assert_feed_indent("*FRS1234.1*", "#", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "##", "** *FRS1234.1.1.1:*")

def assert_double_parse(t, d=5):
    r = list(NumState(d).parse_text(t))
    r2 = list(NumState(d).parse_text("\n".join(r)))
    assert_ab(r, r2)
    
dpt = """*FRS2881.4*
# PBSUsers.conf file will keep track of the following data:
## Per User
### User Name
### Group
### Password
### Email Address
### Date Password was Saved
### Number of Failed Login Attempts
## Per Group
### Group Name
### Password Expiration Period (Days)
## 2-D Permission Table
### Each row corresponds to each group"""
assert_double_parse(dpt)

def assert_new_state(s, exp, nxt=1, ind=0):
    st = NumState()
    for l in s.splitlines():
        st.feed(l)
    for _ in range(nxt):
        st.next()
    for _ in range(ind):
        st.indent()
    assert_ab(st.frs_string(), exp)
    
assert_new_state("*FRS123.1*\n*FRS31.2*", "*FRS31.3:*")
assert_new_state("*FRS123.1*\n*FRS123.3*", "*FRS123.4:*")

def assert_parse_text(txt, exp):
    st = NumState()
    lines = list(st.parse_text(txt))
    res = "\n".join(lines)
    assert_ab(res, exp)

lines = """*FRS1234.1:*
#"""
exp = """*FRS1234.1:*
* *FRS1234.1.1:*"""
assert_parse_text(lines, exp)

lines="""*FRS1234.1:*
#
##
*** Non-numbered FRS text or sub-item
*FRS1234.2:*
Foobar
!Foobar.jpg!
#
##
"""
exp = """*FRS1234.1:*
* *FRS1234.1.1:*
** *FRS1234.1.1.1:*
*** Non-numbered FRS text or sub-item
*FRS1234.2:*
Foobar
!Foobar.jpg!
* *FRS1234.2.1:*
** *FRS1234.2.1.1:*"""

assert_parse_text(lines, exp)

lines = "+*FRS*+\n*FRS1234:*\n#"
exp = "+*FRS*+\n*FRS1234:*\n* *FRS1234.1:*"
assert_parse_text(lines, exp)

def assert_parse_empty(s, exp):
    st = NumState()
    res = None
    for res in st.parse_text(s):
        pass
    assert_ab(res, exp)
    
#assert_parse_empty("*User Requirements*\n# Bar", "# Bar")

In [66]:
lines="""*User Requirements*
* An action should be performed, then the corresponding Alarm record should be modified, and only when both those things have been done successfully should the user event be logged.
* If an alarm is triggered, and the DB is then archived, users should be able to acknowledge that alarm on the RIO without modifying the original alarm record, and a user event should indicate that the alarm was cleared on the RIO but the original record wasn't modified.  The user event should specify the original alarm record ID and DB name.
** At that point, it's possible that Web UI users will still see those alarms in the 'unacknowledged' list (if the Web UI keeps all alarms from the Server in memory), but the number of unacknowledged alarms displayed in the Alarms tab will be 0.  And when the Web UI next refreshes, its memory will clear and it will again only show alarms in the current DB.

*FRS2908.1* Changes to the clearalarm call
# Make the order of events:
## Get the Alarm Name from the DB by looking it up from the ID sent from the Web UI
*** If the record ID is not found in the active DB, or the record in the active DB is not an unacknowledged alarm, check the most recently archived DB.
*** If the lookup also does not work in the most recently archived DB, then generate an error and don't attempt to write/modify any records
## Clear on RIO
## If the original alarm record is in the active DB, modify alarm record
## Write user event record
*** If the original alarm record is in the active DB, the Event Name for the User Event should be modified so it concatenates 3 inputs: 
#### "Acknowledged " (make sure it ends with a space)
#### alarm name (currently used as 1st input to concatenate)
#### " Alarm" (make sure it starts with a space)
*** If the original alarm record is not in the active DB, and was found in the most recently archived DB, the Event Name for the User Event should be "Acknowledged <alarmname> on RIO, but original alarm record <ID> was not modified as the DB <ArchivedDBname> had already been archived."
# Change the query build, and the string format, so its last line is Where ID = whatever *and Alarm_Status = 1* (bold is new)

*FRS2908.1* Changes to the clearalarmsbytype call
# Make the order of events:
## Clear on RIO
## Modify alarm record
## Write user event record
# The Event Name for the User Event should be modified so it concatenates 3 inputs:
## "Acknowledged All " (make sure it ends with a space)
## alarm name (currently used as 1st input to concatenate)
## " Alarms" (make sure it starts with a space)

*FRS2908.2* Changes to the clearallalarms call
# Make the order of events:
## Clear on RIO
## Modify alarm record
## Write user event record
# The Event Name for the User Event should be modified so it is "Acknowledged All Alarms"
"""
res = "\n".join(NumState(3).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)

*User Requirements*
* An action should be performed, then the corresponding Alarm record should be modified, and only when both those things have been done successfully should the user event be logged.
* If an alarm is triggered, and the DB is then archived, users should be able to acknowledge that alarm on the RIO without modifying the original alarm record, and a user event should indicate that the alarm was cleared on the RIO but the original record wasn't modified.  The user event should specify the original alarm record ID and DB name.
** At that point, it's possible that Web UI users will still see those alarms in the 'unacknowledged' list (if the Web UI keeps all alarms from the Server in memory), but the number of unacknowledged alarms displayed in the Alarms tab will be 0.  And when the Web UI next refreshes, its memory will clear and it will again only show alarms in the current DB.

*FRS2908.1* Changes to the clearalarm call
* *FRS2908.1.1:* Make the order of events:
** *FRS